<h1>System of Linear Equations Routines</h1>

In [5]:
# Let us first define the numbers we will work with. Note the use of KK when defining
# both matrices and vectors. Details aside it will make output numbers look nice if you set
# KK equal QQ. Other possible options are RR, CC, QQ, QQbar; here QQbar is another nice option
KK = QQ

def GetEch(M265Matrix):
    # given a matrix defined as above the function returns the echelon form of the matrix
    return M265Matrix.echelon_form()

def HomSol(M265Matrix):
    # obtain the homogeneous solution set of SLE with matrix M265Matrix
    return M265Matrix.right_kernel_matrix()

def ParSol(M265Matrix,M265Vector):
    # obtain a particular solutino to SLE with matrix M265Matrix and vector of constant M265Vector
    (rw,cl)=M265Matrix.dimensions()
    if(rw!=len(M265Vector)):
        print("dimension error")
    try:
        return M265Matrix.solve_right(M265Vector)
    except ValueError:
        #if ValueError occurs there is no solution but instead of raising the error we return
        # the empty set
        return vector(KK,[])
    return False
    
def GetERO(M265Matrix, debug = False):
    # get one list of elemetary operations that reduce a matrix to Reduced Echelon Form
    resEliminations = []
    [Rw,Cl]=M265Matrix.dimensions()
    M265copy = Matrix(KK,M265Matrix)
    if (debug):
        print(M265copy)
    rw = 0
    cl = 0
    for c in range(Cl):
        hasPivot = False
        # first we check if the column under question has a pivot
        for r in range(rw,Rw):
            if (M265copy[r][cl]!=0):
                hasPivot = True
                if (r!=rw):
                    EO = identity_matrix(KK,Rw)
                    EO.swap_rows(rw,r)
                    resEliminations.append(EO)
                    M265copy.swap_rows(rw,r)
                    if (debug):
                        print("\n")
                        print("Swap row ",rw,"with row", r)
                        print(M265copy)
                break # if there is a pivot 
        if (hasPivot):
            # if there is a pivot first scale it to one if the pivot is not one
            if (M265copy[rw][cl]!= KK(1)):
                EO=identity_matrix(KK,Rw)
                EO.rescale_row(rw,KK(1)/M265copy[rw][cl])
                resEliminations.append(EO)
                M265copy=resEliminations[-1]*M265copy
                if (debug):
                    print("\n")
                    print("Scale row", rw)
                    print(M265copy)

            # make the respective entries in the column zero except for the row that contains the pivot (r==rw)
            # further if a row has already entry zero at that column skip that row as the resulting scaling
            # is simply the identity matrix (M265copy[r][cl]==0)
            for r in range(Rw):
                if ((r==rw) or (M265copy[r][cl]==0)):
                    continue
                EO = identity_matrix(KK,Rw)
                EO.add_multiple_of_row(r,rw,-M265copy[r][cl])
                resEliminations.append(EO)
                #M265copy = M265copy.add_multiple_of_row(r,rw,-M265copy[r][cl])
                M265copy=resEliminations[-1]*M265copy
                if(debug):
                    print("\n")
                    print("M265copy multiple of row",rw,"to row",r )
                    print(M265copy)
            rw += 1 # if there was a pivot in this column for the remaining rows up we search for pivots
            # only on rows that are below it
        cl += 1
        
    return resEliminations

<h2>Some <i>nice</i> matrices</h2>

<h3>$3\times 3$ matrices</h3>

In [6]:
M3 = Matrix(KK,[[5,-8,-4],[3,-5,-2],[-4,7,3]])
M4 = Matrix(KK, [\
                 [-3, -2, -8,  1],\
                 [-2,  0, -5,  1],\
                 [ 0, -1, -1,  0],\
                 [ 0, -5,  0, -2]])
print(M3)
print()
print(M4)

[ 5 -8 -4]
[ 3 -5 -2]
[-4  7  3]

[-3 -2 -8  1]
[-2  0 -5  1]
[ 0 -1 -1  0]
[ 0 -5  0 -2]


<h1>Homework</h1>

<h2>Restrictions</h2>
<p>You may run the sage kernel or any other kernel of your choice. However, beyond the Standard Routines provided at the top of this notebook no other function calls are allowed from the sage (or any other library).</p>
<p>If you cannot run sage, please update the above routines with the solutions I've provided for the first homework and use those instead. You are welcome to contact me for further information if that is the case.</p>

<h2>Linear transformations</h2>
Given two matrices $MA$ and $MB$ each of their rows is considered as a vector. Assume MA and MB have the same number of rows. Your implementation tasks are:
<dl>
    <dt>isLinearlyExtendable(MA,MB)</dt>
    <dd>The function isLinearlyExtendable returns True if it is possible to linearly extend the map $\phi(MA[i]) = MB[i]$ to the span of the rows of $MA$; else it returns False.</dd> 
    <dt>matrixForm(MA,MB)</dt>
    <dd>Assume isLinearlyExtendable(MA,MB) returns True. The function matrixForm(MA,MB) should return the matrix form of the map $\phi(MA[i]) = MB[i]$.</dd>
    <dt>isInjection(MA,MB)</dt>
    <dd>Assume isLinearlyExtendable(MA,MB) returns True. The function isBijection(MA,MB) should True if the map $\phi(MA[i]) = MB[i]$ is one to one and False otherwise.</dd>
    <dt>nullSpace(MA,MB)</dt>
    <dd>Assume isLinearlyExtendable(MA,MB) returns True. The function nullSpace(MA,MB) should return a basis for the kernel of the map $\phi(MA[i]) = MB[i]$ in the form of a list where each row is a basis vector; if function is one-to-one it should return an empty list.</dd>
</dl>

In [7]:
def isLinearlyExtendable(MA, MB) -> bool:
    hom_sol_a = HomSol(MA.transpose())
    hom_sol_b = HomSol(MB.transpose())
    
    if hom_sol_a.dimensions()[0] == hom_sol_b.dimensions()[0]:
        return hom_sol_a == hom_sol_b
    elif hom_sol_a.dimensions()[0] < hom_sol_b.dimensions()[0]:
        for row in hom_sol_a:
            if row not in hom_sol_b:
                return False
        return True
    else:
        return False



def matrixForm(MA, MB):
    if isLinearlyExtendable(MA, MB):
        return MB.transpose()
    else:
        return None


def isInjection(MA, MB) -> bool:
    if isLinearlyExtendable(MA, MB):
        res = HomSol(matrixForm(MA, MB))
        if not res:
            return True
        else:
            return False
    else:
        return None


def nullSpace(MA, MB):
    if isLinearlyExtendable(MA, MB):
        if isInjection(MA, MB):
            return []
        else:
            return MB.transpose().right_kernel_matrix(basis = "pivot")
    else:
        return None


In [8]:
determinant_test = Matrix(KK, [[6, 1, 1], [4, -2, 5], [2, 8, 7]])
result = GetERO(determinant_test, True)
result

[ 6  1  1]
[ 4 -2  5]
[ 2  8  7]


Scale row 0
[  1 1/6 1/6]
[  4  -2   5]
[  2   8   7]


M265copy multiple of row 0 to row 1
[   1  1/6  1/6]
[   0 -8/3 13/3]
[   2    8    7]


M265copy multiple of row 0 to row 2
[   1  1/6  1/6]
[   0 -8/3 13/3]
[   0 23/3 20/3]


Scale row 1
[    1   1/6   1/6]
[    0     1 -13/8]
[    0  23/3  20/3]


M265copy multiple of row 1 to row 0
[    1     0  7/16]
[    0     1 -13/8]
[    0  23/3  20/3]


M265copy multiple of row 1 to row 2
[    1     0  7/16]
[    0     1 -13/8]
[    0     0 153/8]


Scale row 2
[    1     0  7/16]
[    0     1 -13/8]
[    0     0     1]


M265copy multiple of row 2 to row 0
[    1     0     0]
[    0     1 -13/8]
[    0     0     1]


M265copy multiple of row 2 to row 1
[1 0 0]
[0 1 0]
[0 0 1]


[
[1/6   0   0]  [ 1  0  0]  [ 1  0  0]  [   1    0    0]
[  0   1   0]  [-4  1  0]  [ 0  1  0]  [   0 -3/8    0]
[  0   0   1], [ 0  0  1], [-2  0  1], [   0    0    1],

[   1 -1/6    0]  [    1     0     0]  [    1     0     0]
[   0    1    0]  [    0     1     0]  [    0     1     0]
[   0    0    1], [    0 -23/3     1], [    0     0 8/153],

[    1     0 -7/16]  [   1    0    0]
[    0     1     0]  [   0    1 13/8]
[    0     0     1], [   0    0    1]
]